In [ ]:
# 功能描述：挂载google drive，点击右侧的 挂载云端硬盘 按钮可以避免重复验证过程
import os
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

In [ ]:
# 自动设置和运行 frpc 客户端
import os
import subprocess

# 创建代码目录
code_dir = '/content/drive/MyDrive/code'
mcp_frp_dir = f'{code_dir}/mcp-frp'

# 创建目录
os.makedirs(code_dir, exist_ok=True)
print(f"确保目录存在: {code_dir}")

# 检查 mcp-frp 是否存在，不存在则下载，存在则更新
if not os.path.exists(mcp_frp_dir):
    print("mcp-frp 目录不存在，正在从 GitHub 下载...")
    os.chdir(code_dir)
    result = subprocess.run(['git', 'clone', 'https://github.com/eatcosmos/mcp-frp.git'], 
                          capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ mcp-frp 下载成功")
    else:
        print(f"❌ 下载失败: {result.stderr}")
        raise Exception("无法下载 mcp-frp")
else:
    print("✅ mcp-frp 目录已存在，正在拉取最新版本...")
    os.chdir(mcp_frp_dir)
    result = subprocess.run(['git', 'pull'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ 代码更新成功")
        if "Already up to date" in result.stdout:
            print("ℹ️ 代码已经是最新版本")
        else:
            print("📦 已拉取最新更改")
    else:
        print(f"⚠️ 更新失败，继续使用现有版本: {result.stderr}")
        # 即使更新失败也继续执行，使用现有版本

# 设置环境变量
from google.colab import userdata
try:
    server_ip = userdata.get('FRPS_SERVER_IP')
    os.environ['FRPS_SERVER_IP'] = server_ip
    print(f"✅ 设置服务器 IP: {server_ip}")
except:
    print("❌ 请在 colab secrets 中设置 FRPS_SERVER_IP")
    raise Exception("需要设置 FRPS_SERVER_IP")

# 先设置 SSH 服务
print("正在设置 SSH 服务...")
os.chdir(mcp_frp_dir)
subprocess.run(['chmod', '+x', 'setupSSH.sh'])

# 运行 setupSSH.sh
ssh_result = subprocess.run(['./setupSSH.sh'], capture_output=True, text=True)
if ssh_result.returncode == 0:
    print("✅ SSH 服务设置成功")
else:
    print("⚠️ SSH 服务设置可能有问题，但继续执行")
    if ssh_result.stderr:
        print(f"SSH 设置输出: {ssh_result.stderr[:200]}...")  # 只显示前200字符

# 运行 frpc 客户端
print("正在启动 frpc 客户端...")
subprocess.run(['chmod', '+x', 'run-frpc.sh'])

# 后台运行 frpc
result = subprocess.run(['./run-frpc.sh', '-d'], capture_output=True, text=True)
print("STDOUT:", result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

if result.returncode == 0:
    print("✅ frpc 客户端已启动")
    print(f"🔗 SSH 连接: ssh -p 7502 root@{server_ip}")
    print("💡 提示: 如果是 colab 环境，用户名通常是 root")
else:
    print("❌ frpc 启动失败")
